Before we begin, let us execute the below cell to display information about the NVIDIA® CUDA® driver and the GPUs running on the server by running the `nvidia-smi` command. To do this, execute the cell block below by clicking on it with your mouse, and pressing Ctrl+Enter, or pressing the play button in the toolbar above. You should see some output returned below the grey cell.

In [ ]:
!nvidia-smi

## Learning objectives
The **goal** of this lab is to:
- Learn how Memory Coherent Architectures like Grace Hopper Superchip help achieve better productivity.
- Understand how applications gain incremental acceleration using chip-to-chip connectivity between CPU and GPU
- We do not intend to cover:
    - Optimization techniques like memory prefetching API and cache eviction policies, which could optimize the performance further for applications.

Let us start by revising some fundamentals covered previously.

**NOTE: This notebook is to run on a Grace Hopper Superchip only.**





## Heterogeneous Computing 

Heterogenous Computing can be defined as combining processors of different types, each specializing in different types of execution.

A Heterogeneous programming model like CUDA includes provisions for a CPU and GPU. These enable developers to target portions of source code for parallel execution on the device (GPU). The programming model provides functions that can be executed on the host (CPU) to interact with the device. The two processors that work with each other are:

- Host: CPU and its memory (Host Memory)
- Device: GPU and its memory  (Device Memory)

  
<img src="../../_common/images/heterogeneous_computing.jpg" width="80%" height="80%">

## Steps in Heterogenous Computing Programming Models

The table below highlights the typical steps that have been required so far to convert sequential codes to target Heterogenous computing architectures using CUDA and OpenACC as an example :

| Sequential code | CUDA Code | OpenACC Code |
| --- | --- | --- |
| **Step 1** Allocate memory on the CPU ( _malloc new_ ) | **Step 1** : Allocate memory on the CPU (_malloc, new_ )| Allocate memory on the CPU (_malloc, new_ ) |
| **Step 2** Populate/initialize the CPU data | **Step 2** Allocate memory on the GPU, using API like _cudaMalloc()_ | **Step 2** Allocate memory on the GPU, using pragma like _#pragma acc data create_ |
| **Step 3** Call the CPU function that has the crunching of data. | **Step 3**  Populate/initialize the CPU  |  **Step 3**  Populate/initialize the CPU  |
| **Step 4** Consume the crunched data on Host | **Step 4** Transfer the data from the host to the device with _cudaMemcpy()_ |  **Step 4** Transfer the data from the host to the device with _#pragma acc data copy_  |
| | **Step 5** Call the GPU function with _<<<,>>>_ brackets | **Step 5** Parallelize and call the GPU function with _#pragma acc parallel loop_ |
| | **Step 6** Synchronize the device and host with _cudaDeviceSynchronize()_ | **Step 6** Synchronize the device and host with _#pragma acc wait_ |
| | **Step 7** Transfer data from the device to the host with _cudaMemcpy()_ | **Step 7** Transfer data from the device to the host with _#pragma acc data copy_ |
| | **Step 8** Consume the crunched data on Host | **Step 8** Consume the crunched data on Host |

CPU and GPU memory are different, and the developer needs to use additional  API to allocate and free memory on GPU. This traditionally created an inertia for developers to adopt heterogeneous computing due to a complex programming model that involves manually managing device memory allocations and data transfer to and from the host.
    
### Unified Memory
With every new CUDA and GPU architecture release, new features are added. These new features provide more performance and ease of programming or allow developers to implement new algorithms that otherwise weren't possible to port on GPUs using CUDA.
One such important feature that was released from CUDA 6.0 onward and finds its implementation from the Kepler GPU architecture is unified memory (UM). 

In simpler words, UM provides the user with a view of a single memory space that's accessible by all GPUs and CPUs in the system. This is illustrated in the following diagram:

<img src="../../_common/images/UM.png" width="60%" height="60%">


UM simplifies programming efforts for beginners as developers need not explicitly manage to copy data to and from GPU. Below is an example usage of how to use managed memory using different programming models covered previously:

<details>
    <summary markdown="span"><b>CUDA C/C++</b></summary>

```cpp
 // Allocate Unified Memory -- accessible from CPU or GPU
  int *a, *b, *c;
  cudaMallocManaged(&a, N*sizeof(int));
  cudaMallocManaged(&b, N*sizeof(int));
  cudaMallocManaged(&c, N*sizeof(int));
  ...

  // Free memory
  cudaFree(a);
  cudaFree(b);
  cudaFree(c);
```
</details>

<details>
    <summary markdown="span"><b>CUDA Fortran</b></summary>
   
```fortran
!matrix data
real, managed, allocatable, dimension(:,:) :: A, B, C
```
</details>


</details>

<details>
    <summary markdown="span"><b>OpenACC</b></summary>
    
In OpenACC we used the Unified Memory feature by enabling the compilation time flag **-acc=gpu -gpu=managed**. OpenACC parallel and loop directives relied on a feature called [CUDA Managed Memory](../../_common/jupyter_notebook/GPU_Architecture_Terminologies.ipynb) to deal with the separate CPU & GPU memories for us. Just adding OpenACC to our loop, we achieved a considerable performance boost without using explicit data copy directives. 

</details>

<details>
    <summary markdown="span"><b>Standard Languages</b></summary>
    
C++ stdpar and DO CONCURRENT in Fortran provide the highest portability and can be seen as the first step to porting on Heterogeneous computing architectures like GPU. The implementation depends on features like Unified memory to make sure the standard languages support varied heterogeneous computing architectures without any vendor-specific functionality.  

</details>

### Memory Coherent Architectures and Unified Memory

Memory coherency is a feature where the multiple processes/threads accessing memory must agree on the state of the memory at all times. For example, thread 0 reads the memory location X, and thread 1 reads the same location at the same time, both threads should always read the same value. But if memory is not coherent, threads A and B might read back different values.

Memory coherency can be provided natively at the hardware level or emulated at the software level. For example, old GPU architecture from NVIDIA supported memory coherency via Unified Memory using a Heterogeneous Memory Management Module, also referred to as [HMM](https://on-demand.gputechconf.com/gtc/2017/presentation/s7764_john-hubbardgpus-using-hmm-blur-the-lines-between-cpu-and-gpu.pdf) feature that use software to emulate memory coherence between CPUs and GPUs. Software-based memory coherency provides its own limitation and is not required in latest generation architectures like Grace Hopper Superchip. The applications transparently benefit from hardware acceleration for memory coherency provided by NVLink-C2C, without any software changes, as described in the next section.

Two main ways to obtain Unified Memory are as follows:

- Fully Supported Unified Memory: Developers allocate memory on the host with system APIs: stack variables, global-/file-scope variables, malloc() / mmap() thread locals, etc.
- Managed Unified Memory: Memory allocated using explicit API, for example, cudaMallocManaged(). This provides backward compatibility and is available on more systems, and may perform better than System-Allocated Memory.

CUDA API provides functions to query the device to check the support for Unified Memory. The following example shows how to detect the Unified Memory support level at runtime: 

```
//Support Full Unified memory like in Grace Hopper SuperChip
cudaDeviceGetAttribute(&pma, cudaDevAttrPageableMemoryAccess, d);
//Support Managed Unified Memory in older architectures 
cudaDeviceGetAttribute(&cma, cudaDevAttrConcurrentManagedAccess, d);
```


In [ ]:
!cd ../source_code && nvcc -o unified_test unified_test.cu && ./unified_test

Let us now delve deeper into the NVIDIA Grace Hopper Superchip architecture to understand the support of Memory Coherency. 

## Grace Hopper Superchip 

The NVIDIA Grace Hopper Superchip architecture was announced in 2023 and is one of the latest architectures in production by NVIDIA. The Superchip brings together the high throughput-based performance of the [NVIDIA Hopper GPU](https://developer.nvidia.com/blog/nvidia-hopper-architecture-in-depth/) with the versatility of the [NVIDIA Grace CPU](https://www.nvidia.com/en-us/data-center/grace-cpu/). The two processors are connected to each other by high bandwidth and memory coherent [NVIDIA NVLink Chip-2-Chip (C2C) interconnect](https://www.nvidia.com/en-us/data-center/nvlink-c2c/) in a single superchip.

<img src="../../_common/images/grace_hopper_arch.jpg" width="80%" height="80%">

While there is much more to the Grace Hopper superchip, this lab focuses on the memory coherency aspect using NVIDIA NVLink-C2C, which is an NVIDIA memory coherent, high-bandwidth, and low-latency superchip interconnect. It is the heart of the Grace Hopper Superchip and delivers up to 900 GB/s total bandwidth. This is 7x higher bandwidth than x16 PCIe Gen5 lanes commonly used in accelerated systems. 

Memory coherency enables developers to transfer only the data needed, and not migrate entire pages to and from the GPU.  NVIDIA NVLink-C2C hardware coherency enables the Grace CPU to cache GPU memory at cache-line granularity and for the GPU and CPU to access each other’s memory without page migrations.

Let us run the Linux Operating System(OS) command to verify that GPU memory is indeed visible to the OS. By running the `numactl` command, ideally you should see the output as follows: 

<img src="../../_common/images/grace_hopper_numactl.jpg" width="80%" height="80%">

The output shows that there are two NUMA nodes on this machine,  and how much memory is available for each node. Please note  the last seven NUMA nodes can be ignored if [MIG](https://www.nvidia.com/en-in/technologies/multi-instance-gpu/) is not used.

In [ ]:
!numactl -H

Let us also use a tool provided by NVIDIA for bandwidth measurements on NVIDIA GPUs. The test measures bandwidth for various memcpy patterns across different links. 

In [ ]:
#!cd /nvbandwidth && ./nvbandwidth -t host_to_device_bidirectional_memcpy_ce
!cp -r -n /nvbandwidth ../source_code/ && cd ../source_code/nvbandwidth && ./nvbandwidth -t host_to_device_bidirectional_memcpy_ce 

For NVIDIA Grace Hopper Superchip ideally we should get the following bi-directional bandwidth between CPU and GPU.

```
Running host_to_device_bidirectional_memcpy_ce.
memcpy CE CPU(row) <-> GPU(column) bandwidth (GB/s)
          0
0    240.64
```

Let us understand in more detail the aspect of access to memory by Grace CPU and Hopper GPU in the next section. 

### Global Access

Grace can read Hopper’s memory and Hopper can read Grace’s memory directly, without moving any data. It’s possible via the NVLink C2C connection. Not only that, but the CPU cache can cache both its memory and the GPU memory, by keeping track of GPU cache lines and CPU cache lines. This means memory accesses from either processor are coherent and the total memory is truly unified.

<img src="../../_common/images/grace_hopper_global_access.jpg" width="75%" height="75%">

- Grace directly reading Hopper’s memory: CPU fetches GPU data into CPU L3 cache. The cache remains coherent with GPU memory. Changes to GPU memory evict cache line.
- Hopper directly reading Grace’s memory: GPU loads CPU data via CPU L3 cache. CPU and GPU can both hit on cached data. Changes to CPU memory update cache line.

Address Translation Service (ATS), as shown in the figure below, enables the CPU and GPU to share a single per-process page table, enabling all CPU and GPU threads to access all system-allocated memory, which can reside on physical CPU or GPU memory. ATS enables the CPU and GPU to share a single per-process page table, enabling all CPU and GPU threads to access all system-allocated memory, which can reside on physical CPU or GPU memory.

<img src="../../_common/images/grace_hopper_ATS.jpg" width="75%" height="75%">

### Memory Allocators Impact Data Placement and Movement

Based on the type of memory allocator and heterogeneous computing architecture being used can impact the data placement and movement. This behaviour can be summarized as follows: 

|  | cudaMalloc | cudaMallocManaged | System API (malloc/mmap/...) |
| --- | --- | --- | --- |
| Placement | GPU | First Touch | First Touch |
| Which Processor Can Access? | GPU | Both CPU and GPU | Both CPU and GPU |
| How does access happen? | GPU MMU | Fault on first access and move page | Direct access over C2C using Automatic Translation Service  |
| Any optimization by Driver? | None | Fault or access counter  | Using access counter to migrate memory between CPU and GPU |

## Application Categories

An analysis is required to estimate the speedup gained while porting and deploying the applications from previous non-memory coherent architectures to memory coherent architectures like Grace Hopper Superchip. Broadly, they can be categorized into the following: 

- **CPU Only Application**:  Applications that work well on latency-reducing architectures like CPU or are yet to be migrated to GPU.
- **Fully GPU Accelerated**: Application which has been majorly ported to GPU and computation happens almost fully on the GPU with data in GPU memory
- **Partially GPU Accelerated**: Application which partially computes on CPU and GPU. They can be further subdivided into two categories
    - **Bound by CPU**: As GPU becomes faster, the majority of the time of application is spent in CPU computation
    - **Bound by CPU<->GPU communication**: Due to back-and-forth data transfers between two different memories, the application hotspot is the transfer and is bottlenecked by the transfer speed.



| | Performance Improvement | Productivity Improvements |
| --- | --- | --- |
| Fully GPU Accelerated | These applications primarily will benefit from the increase in computing power of GPU and minimal performance gain can be seen by using C2C. | Memory-coherent programming approach can improve developer productivity by reducing the learning curve |
| CPU bound Application | These applications primarily will benefit from the increase in computing power of GRACE CPU | Memory-coherent programming approach can improve developer productivity by reducing the learning curve | 
| Bound by CPU<->GPU communication|  These applications will primarily benefit from C2C interconnect, reducing the time to transfer data/pages and using features like ATS | Memory-coherent programming approach can improve developer productivity by reducing the learning curve |

The table above does not consider any aspect of porting code to the latest versatile and energy-efficient ARM-based NVIDIA Grace CPU. We will cover the same separately at the end of this lab.

Let us now understand how to port the application using system API like `malloc` directly and analyze the same using profilers. 

## The Application

This section provides an overview of the scientific problem we focus on and the solver we employ. Then, we execute the single GPU version of the application program.

### Laplace Equation

Laplace Equation is a well-studied linear partial differential equation that governs steady-state heat conduction, irrotational fluid flow, and many other phenomena. 

In this lab, we will consider the 2D Laplace Equation on a rectangle with [Dirichlet boundary conditions](https://en.wikipedia.org/wiki/Dirichlet_boundary_condition) on the left and right boundary and periodic boundary conditions on the top and bottom boundary. We wish to solve the following equation:

$\Delta u(x,y) = 0\;\forall\;(x,y)\in\Omega,\delta\Omega$

### Jacobi Method

The Jacobi method is an iterative algorithm to solve a linear system of strictly diagonally dominant equations. The governing Laplace equation is discretized and converted to a matrix amenable to Jacobi-method-based solver. The pseudo-code for the Jacobi iterative process can be seen in the diagram below:

<img src="../../_common/images/jacobi_algo.jpg" width="90%" height="90%">


The outer loop defines the convergence point, which could either be defined as reaching the max number of iterations or when [L2 Norm](https://link.springer.com/referenceworkentry/10.1007%2F978-0-387-73003-5_1070) reaches a max/min value. 


### The Code

The GPU processing flow, in general, follows three key steps:

1. Copy data from CPU to GPU
2. Launch GPU Kernel
3. Copy processed data back to the CPU from the GPU

<img src="../../_common/images/gpu_programming_process.png" width="70%" height="70%">

We follow the same three steps in our code. Let's understand the single GPU code first. 

The source code is available at [jacobi.cu](../source_code/single_gpu/jacobi.cu) (click to open). Similarly, have a look at the [Makefile](../source_code/single_gpu/Makefile). 

Refer to the `single_gpu(...)` function. The important steps at each iteration of the Jacobi Solver inside `while` loop are:
1. The norm is set to 0 using `cudaMemset`.
2. The device kernel `jacobi_kernel` is called to update the interior points.
3. The norm is copied back to the host using `cudaMemcpy` (DtoH), and
4. The periodic boundary conditions are re-applied for the next iteration using `cudaMemcpy` (DtoD).

```
    while (l2_norm > tol && iter < iter_max) {
        cudaMemset(l2_norm_d, 0, sizeof(float));

	   // Compute grid points for this iteration
        jacobi_kernel<<<dim_grid, dim_block>>>(a_new, a, l2_norm_d, iy_start, iy_end, nx);
       
        cudaMemcpy(l2_norm_h, l2_norm_d, sizeof(float), cudaMemcpyDeviceToHost));

        // Apply periodic boundary conditions
        cudaMemcpy(a_new, a_new + (iy_end - 1) * nx, nx * sizeof(float), cudaMemcpyDeviceToDevice);
        cudaMemcpy(a_new + iy_end * nx, a_new + iy_start * nx, nx * sizeof(float),cudaMemcpyDeviceToDevice);

	    cudaDeviceSynchronize());
	    l2_norm = *l2_norm_h;
	    l2_norm = std::sqrt(l2_norm);

        iter++;
	    if ((iter % 100) == 0) printf("%5d, %0.6f\n", iter, l2_norm);
        std::swap(a_new, a);
    }
```

Note that we run the Jacobi solver for 1000 iterations over the grid. The code is present and can be studied here: [C/C++ version](../source_code/jacobi.cu) and [Makefile](../source_code/Makefile). Let's compile the code by running the below cells.


In [ ]:
!cd ../source_code && make clean && make && make run

The output reports the norm value every 100 iterations and the total execution time of the Jacobi Solver. The expected output is:

```
Single GPU jacobi relaxation: 1000 iterations on 16384 x 16384 mesh
    0, 31.999022
  100, 0.897983
  200, 0.535684
  300, 0.395651
  400, 0.319039
  500, 0.269961
  600, 0.235509
  700, 0.209829
  800, 0.189854
  900, 0.173818
16384x16384: 1 GPU:   4.4512 s
```

The execution time may differ depending on the GPU, but the norm value after every 100 iterations should be the same. The program accepts `-nx` and `-ny` flags to change the grid size (preferably a power of 2) and `-niter` flag to change the number of iterations.

### Profiling analysis

Before we profile the code, we would need to configure the Nsight Systems to capture C2C events. To achieve this, we use the following option:

- `--event-sample=system-wide`, collects system-wide event samples.
- `--cpu-socket-events='comma separated events'`, collects per-socket Uncore PMU counters.

We choose a few events to show addional information with respect to C2C events. For more details on the events available, run the following cells: 

In [ ]:
!nsys profile --cpu-socket-events=help:All

Below is an example output for the chosen events.

```
'61'	Socket_0_C2C0/rd_bytes_local
	In a system with the GPU connected to the SoC via the C2C bus, counts the bytes read via ATS and Extended
	GPU Memory traffic by the Socket 0 GPU to the Socket 0 CPU memory.
	In a system with two SoCs connected via a C2C bus, counts the bytes read via remote socket PCIe traffic to
	the Socket 0 CPU memory.

'67'	Socket_0_C2C0/total_bytes_local
	In a system with the GPU connected to the SoC via the C2C bus, counts the bytes read and written via ATS
	and Extended GPU Memory traffic by the Socket 0 GPU to the Socket 0 CPU memory.
	In a system with two SoCs connected via a C2C bus, counts the bytes read and
	relaxed-ordered bytes written via remote socket PCIe traffic to the Socket 0 CPU memory.

'69'	Socket_0_C2C0/total_requests_local
	In a system with the GPU connected to the SoC via the C2C bus, counts the read and write requests via
	ATS and Extended GPU Memory traffic by the Socket 0 GPU to the Socket 0 CPU memory.
	In a system with two SoCs connected via a C2C bus, counts the read requests and relaxed-ordered write
	requests via remote socket PCIe traffic to the Socket 0 CPU memory.

'71'	Socket_0_C2C0/wr_bytes_local
	In a system with the GPU connected to the SoC via the C2C bus, counts the bytes written via ATS and Extended
	GPU Memory traffic by the Socket 0 GPU to the Socket 0 CPU memory.
	In a system with two SoCs connected via a C2C bus, counts the relaxed-ordered bytes written via remote socket
	PCIe traffic to the Socket 0 CPU memory.
```

Below command shows the list of available CPU core events that can be and the maximum number of CPU events that can be sampled concurrently.

In [ ]:
!nsys profile --cpu-core-events=help

Now, let's profile the code using NVIDIA Nsight Systems.

In [ ]:
!cd ../source_code && nsys profile --flush-on-cudaprofilerstop=false --gpu-metrics-device=0 --gpu-metrics-frequency=20000 --trace=cuda,nvtx --event-sample=system-wide --cpu-socket-events='61,67,69,71' -o jacobi_report --force-overwrite true ./jacobi

Let's checkout the profiler's report.  Download and save the report file by holding down <mark>Shift</mark> and <mark>right-clicking</mark> the [report](../source_code/jacobi_report.nsys-rep)  then choosing <mark>save Link As</mark>. Once done, open it via the GUI. Below is an example screenshot on the Grace Hopper superchip. It shows the collected uncore events in their own row (see red box in the screenshot). It counts the bytes read, written, the total bytes read and written as well as the counts of read and write requests via ATS. Hovering the cursor over an event sampling row in the timeline shows the event’s rate at that moment.

<img src="../../_common/images/jacobi_uncore.png" width="70%" height="70%">

The second screenshot shows the CUDA API row and the kernel. CUDA API shows traces of CUDA Runtime calls made by the application (e.g: cudaMalloc).

<img src="../../_common/images/jacobi_cuda_api.png" width="70%" height="70%">

## Lab Task

The code requires explicit memory copy API like **cudaMemcpy**.  The task in this section is to modify the source code [C/C++ version](../source_code/jacobi.cu) and remove any explicit memory allocation and memory copy API like **cudaMalloc** and **cudaMemcpy** . Replace the same with malloc and pass the same pointer to both CPU and GPU function calls. 

Compile and profile the code to analyze the timeline provided by the profiler. Note the absence of explicit calls to memory copy APIs.

In [ ]:
!cd ../source_code && make clean && make
!cd ../source_code && nsys profile --flush-on-cudaprofilerstop=false --gpu-metrics-device=0 --gpu-metrics-frequency=20000 --trace=cuda,nvtx --event-sample=system-wide --cpu-socket-events='61,67,69,71' -o jacobi_report --force-overwrite true ./jacobi


Feel free to checkout the solutions available at [C/C++ version](../source_code/jacobi_solution.cu) to help you understand better. Download and save the report file by holding down <mark>Shift</mark> and <mark>right-clicking</mark> the [report](../source_code/jacobi_report.nsys-rep)  then choosing <mark>save Link As</mark>. Once done, open it via the GUI. 

Below is an example screenshot on the Grace Hopper superchip. It shows the collected uncore events in their own row (see red box in the screenshot). When we replace the `cudaMalloc` calls with `malloc`, we can see more C2C activities comparing to when we use `cudaMalloc` to allocate memory on device. 

<img src="../../_common/images/jacobi_solution_uncore.png" width="70%" height="70%">

The below example screenshot shows that we have lots of `cudaMemcpy` as the data that was allocated using `malloc` is accessed directly by the GPU via using Automatic Translation Service. 

<img src="../../_common/images/jacobi_solution_cuda_api.png" width="70%" height="70%">


## API Change and Coding Guidelines

Memory coherent architecture like Grace Hopper Superchip strives to make sure there is:
- No programming model changes
- No new APIs to be learned
- No changes to existing APIs

For example, the Unified Memory Programming feature used in the previous lectures of OpenACC, CUDA, and Standard Languages is supported for all platforms. Which primarily means memory accesses just work. 

Additionally,  hints can be provided and only impact performance, not results.
- [cudaMemAdvise](https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__MEMORY.html#group__CUDART__MEMORY_1ge37112fc1ac88d0f6bab7a945e48760a): PreferredLocation, AccessedBy.
- [cudaMemPrefetch](https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__MEMORY.html#group__CUDART__MEMORY_1ge8dc9199943d421bc8bc7f473df12e42): prefetch to NUMA node.


## CPU only

OpenACC, Standard Languages  and OpenMP are supposed to work out of the box.

### Compilers Support for Grace:
**NVHPC**: 
- Focused on application performance,
- High velocity, constant innovation

**LLVM and Clang:**
- NVIDIA provides builds of [Clang](https://developer.nvidia.com/grace/clang) for Grace
- Drop-in replacement for mainline Clang
- 100% of Clang enhancements for Grace are contributed to mainline LLVM

**GCC**
- NVIDIA contributes to mainline GCC to support Grace
- Working with all major Linux distros to improve the availability of Grace optimizations in GCC

#### Tips for porting from x86 to ARM CPU like 
- Remove all architecture-specific flags: `-mavx`,` -mavx2`, etc.
- Remove `-march` and `-mtune` flags. 
- Use `-Ofast -mcpu=native`
- If fast math optimizations are not acceptable, use `-O3 –ffp-contract=fast`
- Use `–flto` to enable link-time optimization 
- Apps may need `-fsigned-char` or `-funsigned-char` depending on the developer’s assumption
- `gfortran` may benefit from `-fno-stack-arrays`

[Nvidia Performance Libraries](www.developer.nvidia.com/nvpl) allows easy porting of applications by providing drop-in replacement for any math library implementing standard interfaces (e.g. Netlib, FFTW).

```
gcc -DUSE_CBLAS -ffast-math -mcpu=native -O3 -I/PATH/TO/nvpl/include -L/PATH/TO/nvpl/lib -o mt-dgemm.nvpl mt-dgemm.c -lnvpl_blas_lp64_gomp

```
Other options include using libraries and frameworks like ATLAS, OpenBLAS, BLIS, which are community-supported with some optimizations. They work for Grace, but are unlikely to outperform NVPL. 


### SIMD 

ARM assembly is simpler than x86. Arm processors have a much simpler and general set of registers than x86. Just assign a one-to-one mapping from an x86 register to an Arm register when porting code. Complex x86 instructions will become multiple Arm instructions.

Follow Arm’s documentation on rewriting x86 vector intrinsic:
- SVE porting: https://developer.arm.com/documentation/101726/latest
- NEON Porting: https://developer.arm.com/documentation/101725/0300/Coding-for-Neon 


# Links and Resources
[NVIDIA Hopper GPU](https://developer.nvidia.com/blog/nvidia-hopper-architecture-in-depth/) 

[NVIDIA Grace CPU](https://www.nvidia.com/en-us/data-center/grace-cpu/)

[NVIDIA Nsight System](https://docs.nvidia.com/nsight-systems/)


**NOTE**: To be able to see the Nsight Systems profiler output, please download the latest version of Nsight Systems from [here](https://developer.nvidia.com/nsight-systems).

Don't forget to check out additional [Open Hackathons Resources](https://www.openhackathons.org/s/technical-resources) and join our [OpenACC and Hackathons Slack Channel](https://www.openacc.org/community#slack) to share your experience and get more help from the community.

--- 

## Licensing 

Copyright © 2022 OpenACC-Standard.org.  This material is released by OpenACC-Standard.org, in collaboration with NVIDIA Corporation, under the Creative Commons Attribution 4.0 International (CC BY 4.0). These materials may include references to hardware and software developed by other entities; all applicable licensing and copyrights apply.